In [1]:
import os
import requests
import asyncio

import math
import rasterio
import matplotlib.pyplot as plt
import planet
from planet import Session

from requests.auth import HTTPBasicAuth

In [50]:
#DOWNLOAD_DIR = "~/Documents/Modules/Thesis/lake_victoria/data/"

DOWNLOAD_DIR = os.getenv('~/Documents/Modules/Thesis/lake_victoria/data/', '.')

In [51]:
def create_requests():
    # The Orders API will be asked to mask, or clip, results to
    # this area of interest.
    usenge_aoi = {
        "type":
        "Polygon",
        "coordinates": [[[34.058098,-0.11658900000000472],
                         [34.10126,-0.11658900000000472],
                         [34.10126,-0.07127399999998829],
                         [34.058098,-0.07127399999998829],
                         [34.058098,-0.11658900000000472]]]
    }

    # In practice, you will use a Data API search to find items, but
    # for this example take them as given.
    usenge_items = ['5740619_3639020_2022-06-26_2442', '5740619_3639019_2022-06-26_2442']

    usenge_order = planet.order_request.build_request(
        name='usenge_order',
        products=[
            planet.order_request.product(item_ids=usenge_items,
                                         product_bundle='analytic_udm2',
                                         item_type='PSOrthoTile')
        ],
        tools=[planet.order_request.clip_tool(aoi=usenge_aoi)])
    mfangano_aoi = {
        "type":
        "Polygon",
        "coordinates": [[[34.038471,-0.4648899999999969],
                     [34.091724,-0.4648899999999969],
                     [34.091724,-0.4226070000000135],
                     [34.038471,-0.4226070000000135],
                     [34.038471,-0.4648899999999969]]]
    }
    mfangano_items = ['5764977_3638819_2022-07-06_247a',
                  '5764977_3638820_2022-07-06_247a',
                  '5764977_3638919_2022-07-06_247a']
    mfangano_order = planet.order_request.build_request(
    name='mfangano_order',
    products=[
        planet.order_request.product(item_ids=mfangano_items,
                                     product_bundle='analytic_udm2',
                                     item_type='PSOrthoTile')
    ],
    tools=[planet.order_request.clip_tool(aoi=mfangano_aoi)])
    return [usenge_order, mfangano_aoi] 

In [60]:
async def create_and_download(client, order_detail, directory):
    """Make an order, wait for completion, download files as a single task."""
    with planet.reporting.StateBar(state='creating') as reporter:
        order = await client.create_order(order_detail)
        reporter.update(state='created', order_id=order['id'])
        await client.wait(order['id'], callback=reporter.update_state)

    await client.download_order(order['id'], directory, progress_bar=True)


In [59]:
async def main():
    async with planet.Session() as sess:
        client = sess.client('orders')
        print(client)
        requests = create_requests()
        
        await asyncio.gather(*[
            create_and_download(client, request, DOWNLOAD_DIR)
            for request in requests
        ])


asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop